In [1]:
import cv2
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd

# CATEGORIES = ["boxing", "handclapping", "handwaving", "jogging", "running", 
#     "walking"]


In [ ]:
df = pd.read_csv("../data_generated/list_boxing_person01_s1.csv")
df.shape
df.iloc[0,0:384]

In [2]:
#TRAIN_PEOPLE_ID = [11, 12, 13, 14, 15, 16, 17, 18]
#DEV_PEOPLE_ID = [19, 20, 21, 23, 24, 25, 1, 4]
#TEST_PEOPLE_ID = [22, 2, 3, 5, 6, 7, 8, 9, 10]
TRAIN_PEOPLE_ID = [1, 2, 3, 4]
TEST_PEOPLE_ID = [5]

In [ ]:
data_gen_path = "../data_generated"
#os.listdir(data_gen_path)
no_cuboids = 48
feature_lenght = 192
feature_lenght_all = 250

def concatenate_features(path, flac_all=True):
    files = os.listdir(path)
    angles = []
    magnitudes = []
    all_f = []
    
    train = []
    test = []
    print('Nº files:', len(files))
    
    i=0
    for file in files:
        person_id = int(file.split("_")[2][7:])
#        print(person_id)
        features = []
        pt = os.path.join(data_gen_path,file)
        print('processing video_csv: ',i," file:",pt)
        
        i=i+1
        
        df = pd.read_csv(pt)
        for i in range(no_cuboids): #for each row
            if flac_all:
                angles_magnitudes = list(df.iloc[i,0:feature_lenght_all])
                all_f.append(angles_magnitudes)
                features.append(np.array(all_f))
#             else: 
#                 f_angles = list(df.iloc[i,0:feature_lenght])#first 192 columns
#                 f_magnitudes = list(df.iloc[i,feature_lenght:2*feature_lenght])  #last 192 columns
#                 angles.append(f_angles)
#                 magnitudes.append(f_magnitudes)
#                 features.append(np.array(f_magnitudes))
        
        if person_id in TRAIN_PEOPLE_ID:
            train.append({
                "video": file,
                "category": df.activity[0], 
                "features": features
            })
        else: test.append({
                "video": file,
                "category": df.activity[0], 
                "features": features
            })
        
        
    
#     print('feature len:', len(angles))
    print('train len:', len(train))
    print('test len:', len(test))
    return train, test, angles, magnitudes

train, test ,angles,magnitudes = concatenate_features(data_gen_path)
pickle.dump(train, open("../pickle/train.p", "wb"))
pickle.dump(test, open("../pickle/test.p", "wb"))


Nº files: 121
processing video_csv:  0  file: ../data_generated/list_walking_person05_s3.csv
processing video_csv:  47  file: ../data_generated/list_handclapping_person03_s2.csv
processing video_csv:  47  file: ../data_generated/list_running_person01_s4.csv
processing video_csv:  47  file: ../data_generated/list_jogging_person04_s3.csv
processing video_csv:  47  file: ../data_generated/list_jogging_person01_s4.csv
processing video_csv:  47  file: ../data_generated/list_handwaving_person03_s4.csv
processing video_csv:  47  file: ../data_generated/list_boxing_person04_s2.csv
processing video_csv:  47  file: ../data_generated/list_running_person04_s2.csv
processing video_csv:  47  file: ../data_generated/list_running_person02_s4.csv
processing video_csv:  47  file: ../data_generated/list_walking_person01_s2.csv
processing video_csv:  47  file: ../data_generated/list_boxing_person03_s2.csv
processing video_csv:  47  file: ../data_generated/list_boxing_person02_s2.csv
processing video_csv: 

In [ ]:
train[0]["features"]

In [ ]:
clusters = 100

from sklearn.cluster import KMeans
from numpy import size

# Clustering with KMeans.
print("Running KMeans clustering")
kmeans = KMeans(init='k-means++', n_clusters=clusters, n_init=10, n_jobs=2,
    verbose=1)
kmeans.fit(angles)
# Save trained kmeans object to file.
pickle.dump(kmeans, open("../pickle/cb_%dclusters.p" % clusters, "wb"))

## Building Bag of Words

In [ ]:
from scipy.cluster.vq import vq

def make_bow(dataset, clusters):
    n_videos = len(dataset)
    bow = np.zeros((n_videos, clusters.shape[0]), dtype=np.float)
    # Make bow vectors for all videos.
    video_index = 0
    for video in dataset:
        visual_word_ids = vq(video["features"], clusters)[0]
        #print(len(visual_word_ids))
        for word_id in visual_word_ids:
            bow[video_index, word_id] += 1
        video_index += 1
        
    # Check whether to use TF-IDF weighting.
    tfidf = False
    if tfidf:
        print("Applying TF-IDF weighting")
        freq = np.sum((bow > 0) * 1, axis = 0)
        idf = np.log((n_videos + 1) / (freq + 1))
        bow = bow * idf

    # Replace features in dataset with the bow vector we've computed.
    video_index = 0
    for i in range(len(dataset)): #for each video
        dataset[i]["features"] = bow[video_index]
        video_index += 1

        #if (i + 1) % 50 == 0:
        #    print("Processed %d/%d videos" % (i + 1, len(dataset)))
    print("reeplaced features by bow...")
    return dataset


dataset_train  = "../pickle/train.p"
dataset_test  = "../pickle/test.p"
codebook_file = "../pickle/cb_100clusters.p"
# Load clusters.
codebook = pickle.load(open(codebook_file, "rb"))
clusters = codebook.cluster_centers_

# Load dataset.
dataset_train = pickle.load(open(dataset_train, "rb"))
dataset_test = pickle.load(open(dataset_test, "rb"))

train_bow = make_bow(dataset_train, clusters)
test_bow = make_bow(dataset_test, clusters)
#len(test_bow), train_bow

In [ ]:
clusters[0].shape, len(dataset_train[0]["features"])

In [ ]:
dataset_train[0]["features"].shape

## Train

In [ ]:
from sklearn.svm import SVC

def make_dataset(dataset):
    X = []
    Y = []

    for video in dataset:
        X.append(video["features"])
        Y.append(video["category"])
    return X,Y
        
X_train, Y_train = make_dataset(dataset_train)
X_test, Y_test = make_dataset(dataset_test)

In [ ]:
X_test

In [ ]:
# Train SVM and save to file.
clf = SVC(C=1, kernel="linear", verbose=True)
clf.fit(X_train, Y_train)

## Prediction

In [ ]:
correct = 0
for video in dataset_train:    
    #video = dataset[0]
    predicted = clf.predict([video["features"]])
    print(predicted)
    # Check if majority is correct.
    if predicted == video["category"]:
        correct += 1
print("%d/%d Correct" % (correct, len(dataset_train)))
print("Accuracy =", correct / len(dataset_train))

In [ ]:
Y_test

In [ ]:
Y

## train

In [ ]:
from sklearn.svm import SVC

def make_dataset(dataset):
    X = []
    Y = []

    for video in dataset:
        X.append(video["features"])
        Y.append(video["category"])
    return X,Y
        
X, Y = make_dataset(dataset)


In [ ]:
X

In [ ]:
# Train SVM and save to file.
clf = SVC(C=1, kernel="linear", verbose=True)
clf.fit(X, Y)

In [ ]:
X_test = dataset[1]['features']
Y_test = dataset[1]['category']

In [ ]:
X_test = X
Y_test = Y

In [ ]:
Y_pred = clf.predict(X)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_pred, Y_test)